In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [2]:
def average(df):
    sum = 0
    count = 0
    for x in df['f1']:
        if x!=-1:
            sum+=x
            count+=1
    avg = sum/count
    return avg


In [3]:
#Timer function
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [4]:
def results(model, X_train, X_valid,y_train, y_valid):
    soft_probs = model.predict_proba(X_valid)
    valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
    print(valid_auc)
    train_preds = model.predict(X_train)
    acc = accuracy_score(y_train, train_preds)
    print(acc)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    print(acc)
    print(classification_report(y_valid,preds))

In [5]:
df = pd.read_csv("train_final.csv")
df.describe()
# f1_avg = average(df)
# df['f1'].replace(-1,f1_avg, inplace = True)
# df.describe()

,Id,Y,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
count,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,...,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,1.638300e+04,16383.000000
mean,8192.000000,0.942135,43007.775865,1.044375,11.770938,118323.581456,1.044436,0.050052,117089.674113,169730.178600,...,25894.316914,119045.099005,184622.040835,1.047305,125959.667765,1.044558,1.045718,1.041934,3.271890e+04,1.043948
std,4729.509065,0.233495,33611.182771,0.264806,353.187115,4518.059755,0.265601,0.293892,10261.292970,69396.677853,...,36086.993946,18321.987129,100590.811845,0.306239,31091.344158,0.262576,0.266874,0.246597,3.184929e+06,0.259640
min,1.000000,0.000000,-1.000000,1.000000,1.770000,23779.000000,1.000000,0.000000,4292.000000,4673.000000,...,25.000000,4674.000000,3130.000000,1.000000,117879.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
25%,4096.500000,1.000000,20311.000000,1.000000,1.770000,118096.000000,1.000000,0.000000,117961.000000,117906.000000,...,4554.000000,118395.000000,118398.000000,1.000000,118274.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
50%,8192.000000,1.000000,35527.000000,1.000000,1.770000,118300.000000,1.000000,0.000000,117961.000000,128130.000000,...,13234.000000,118929.000000,119095.000000,1.000000,118568.000000,1.000000,1.000000,1.000000,2.000000e+00,1.000000
75%,12287.500000,1.000000,74240.500000,1.000000,3.540000,118386.000000,1.000000,0.000000,117961.000000,234498.500000,...,38902.000000,120539.000000,290919.000000,1.000000,120006.000000,1.000000,1.000000,1.000000,9.000000e+00,1.000000
max,16383.000000,1.000000,312152.000000,7.000000,43910.160000,286791.000000,9.000000,10.000000,311178.000000,311867.000000,...,311696.000000,286792.000000,308574.000000,18.000000,311867.000000,8.000000,8.000000,7.000000,4.042886e+08,8.000000


In [6]:
y = df.loc[:,'Y']
X = df.loc[:,'f1':'f24']
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size = 0.75, test_size = 0.25,random_state = 42, shuffle = True)

(colsample_bytree=0.3,subsample = 0.7,max_depth=8,
                            n_estimators=1550, learning_rate =0.011,
                            colsample_bylevel=0.5,n_jobs=-1,base_score = 0.55,
                            random_state=42,)
                     (colsample_bytree=0.4,subsample = 1,max_depth=7,
                            n_estimators=1350, learning_rate =0.012,
                            colsample_bylevel=0.6,n_jobs=-1,base_score = 0.55,
                            random_state=42)
 (colsample_bytree=0.395,subsample = 0.8667,max_depth=10,
                            n_estimators=1550, learning_rate =0.0108,
                            colsample_bylevel=0.65,n_jobs=-1,base_score = 0.475,
                            random_state=42) 
 (colsample_bytree=0.395,subsample = 0.8667,max_depth=11,
                            n_estimators=1538, learning_rate =0.0105,colsample_bynode = 0.87,
                            colsample_bylevel=0.65,n_jobs=-1,base_score = 0.475,
                            random_state=42, max_delta_step =1.168)

In [7]:
params={'colsample_bylevel': 0.8789900701708724,
 'colsample_bynode': 0.9962857441883629,
 'colsample_bytree': 0.7448296344616587,
 'learning_rate': 0.036130784427853585,
 'max_delta_step': 4.953440861348021,
 'max_depth': 9,
 'n_estimators': 980,
 'subsample': 0.6002618478477451}

In [8]:
model_trial = XGBClassifier  (
                           n_jobs=-1,base_score = 0.475,
                            random_state=42,
                            **params
                             )
model_trial.fit(X_train, y_train)
results(model_trial,X_train, X_valid, y_train, y_valid)

0.8660956408648169
1.0
0.9609375
              precision    recall  f1-score   support

           0       0.92      0.31      0.47       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.94      0.66      0.72      4096
weighted avg       0.96      0.96      0.95      4096



Although the auc score is slighly less, the precision and recal is slightly better than the current best model. Lets make a submission to see if its worth relying on precision/recall. Yes, I think it is reliable as it did not break it. 
Update: Making a new submission after adding new parameter max_delta_step

In [58]:
model_trial.fit(X,y)

XGBClassifier(base_score=0.475, booster='gbtree', colsample_bylevel=0.65,
              colsample_bynode=0.87, colsample_bytree=0.395, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.0105, max_delta_step=1.168, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1538, n_jobs=-1, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8667, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [59]:
testdf = pd.read_csv("test_final.csv")
testdf['f1'].replace(-1,f1_avg, inplace = True)
X_test = testdf.loc[:,'f1':'f24']
test_preds = model_trial.predict_proba(X_test)
sub10 = {"Id":testdf['Id'],"Y":test_preds[:,1]}
sub10= pd.DataFrame(data=sub10)
sub10.to_csv("submissions/sub10.csv", index=False)

 Slight improvement noticed in LB. Score now is 0.90633 Stack with lightgbm now
 EDIT: Sub8 overwrote sub6 but its ok. Dont commit changes to sub6. And score increased. Updating current best xgb params.
 EDIT: Testing sub9 with greater reliance on the randomized cross validation results instead of single validation score. This test gave worse results on Leaderboard. 

# Thorough Randomized search with our current best XGBoost
Goal to achieve validation auc of 0.9 before next submission!

In [40]:
params ={
             'min_child_weight':[1e-2, 1e-1, 1,],
             'max_delta_step':sp_uniform(loc=0.5,scale=1.5)
        }

In [41]:
model_trial = XGBClassifier(colsample_bytree=0.395,subsample = 0.8667,max_depth=11,
                            n_estimators=1538, learning_rate =0.0105,colsample_bynode = 0.87,
                            colsample_bylevel=0.65,n_jobs=-1,base_score = 0.475,
                            random_state=42, max_delta_step =1.2)

In [42]:
folds = 5
param_comb = 20 #number of random parameter combos to pick
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 43)
random_search = RandomizedSearchCV(model_trial, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf.split(X_train,y_train), 
                                   verbose=3, random_state=42 )
start_time = timer(None) # timing starts from this point for "start_time" variable
tuned = random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable
results(tuned,X_train, X_valid, y_train, y_valid)
print(tuned.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.3min finished



 Time taken: 0 hours 10 minutes and 38.99 seconds.
0.883110541801948
0.9985350370310084
0.960205078125
              precision    recall  f1-score   support

           0       0.96      0.29      0.44       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.96      0.64      0.71      4096
weighted avg       0.96      0.96      0.95      4096

{'max_delta_step': 0.7751521847992457, 'min_child_weight': 0.01}


# LGBMClassifier Ahead


In [18]:

model_lgb=LGBMClassifier(colsample_bytree=0.9,subsample =0.5,max_depth=-1,
                            n_estimators=5000, learning_rate =0.01)

In [22]:
params ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [28]:
folds = 5
param_comb = 25 #number of random parameter combos to pick
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 43)
random_search = RandomizedSearchCV(model_lgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf.split(X_train,y_train), 
                                   verbose=3, random_state=42 )
start_time = timer(None) # timing starts from this point for "start_time" variable
tuned = random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  3.1min finished



 Time taken: 0 hours 3 minutes and 16.93 seconds.


In [26]:
results(tuned,X_train, X_valid, y_train, y_valid)

0.8597658555932703
0.9858386912997477
0.96044921875
              precision    recall  f1-score   support

           0       0.89      0.32      0.47       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.92      0.66      0.72      4096
weighted avg       0.96      0.96      0.95      4096



In [27]:
tuned.best_params_

{'colsample_bytree': 0.6640914962437607,
 'min_child_samples': 363,
 'min_child_weight': 0.01,
 'num_leaves': 19,
 'reg_alpha': 0,
 'reg_lambda': 5,
 'subsample': 0.3457888702304499}

In [84]:
model_lgb=LGBMClassifier(colsample_bytree=0.9,subsample =0.3,max_depth=-1,
                            n_estimators=1850, learning_rate =0.01)
model_lgb.fit(X_train,y_train)
results(model_lgb,X_train, X_valid, y_train, y_valid)

[LightGBM] [Warning] bagging_fraction is set=0.331, subsample=0.5 will be ignored. Current value: bagging_fraction=0.331
0.8509583456316412
0.9598762920159518
0.9619140625
              precision    recall  f1-score   support

           0       1.00      0.30      0.47       224
           1       0.96      1.00      0.98      3872

    accuracy                           0.96      4096
   macro avg       0.98      0.65      0.72      4096
weighted avg       0.96      0.96      0.95      4096

